In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
input_str = 'apple'      # 입력 문자열
label_str = 'pple!'      # 예측 대상(라벨) 문자열 — 한 글자씩 오른쪽으로 shift된 형태

# 문자 집합(vocabulary) 생성
# 1. input_str과 label_str을 합친 후 → 'applepple!'
# 2. set으로 중복 문자 제거 → {'a', 'p', 'l', 'e', '!'}
# 3. list로 변환하고 정렬 → ['!', 'a', 'e', 'l', 'p']
char_vocab = sorted(list(set(input_str + label_str)))

print(char_vocab)  # 생성된 문자 집합 출력 → ['!', 'a', 'e', 'l', 'p']

# 문자 집합의 크기 계산 (→ 예측해야 할 클래스 개수 = 출력 차원 수)
vocab_size = len(char_vocab)
print(vocab_size)  # 예: 5

['!', 'a', 'e', 'l', 'p']
5


In [3]:
input_size = 5       # 입력 차원 크기 (one-hot 벡터 크기 = 문자의 개수 = 5)
output_size = 5      # 출력 차원 크기 (예측해야 할 문자의 개수 = 5)

hidden_size = 5      # RNN의 hidden state 차원 (RNN 내부에서 사용하는 메모리 용량 같은 것)
                    # 너무 작으면 기억을 잘 못하고, 너무 크면 과적합 위험

learning_rate = 0.1  # 학습률 (Optimizer가 손실을 얼마나 빠르게 줄여나갈지 조절)

In [4]:
# 각 문자(char)를 고유한 숫자(index)로 매핑하는 딕셔너리 생성 (문자 → 인덱스)
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))

# 숫자(index)를 다시 문자(char)로 매핑하는 딕셔너리 생성 (인덱스 → 문자)
index_to_char = dict((i, c) for i, c in enumerate(char_vocab))

# 두 매핑 딕셔너리 출력 (확인용)
print(char_to_index)
print(index_to_char)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [5]:
# input_str = 'apple'의 각 문자들을 숫자 인덱스로 변환 → [1, 4, 4, 3, 2] (예시)
x_data = [char_to_index[c] for c in input_str]

# label_str = 'pple!'의 각 문자들도 인덱스로 변환 → [4, 4, 3, 2, 0] (예시)
y_data = [char_to_index[c] for c in label_str]

# RNN 입력은 (배치 크기, 시퀀스 길이) 형태이므로
# 데이터를 배치 차원을 갖는 2차원 리스트로 변형함
# 즉, [[1, 4, 4, 3, 2]]와 같이 바꿔줌 (배치 크기: 1)
x_data = [x_data]
y_data = [y_data]

# 변환 결과 출력 (확인용)
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [6]:
# np.eye(vocab_size): 단위 행렬 생성 (예: vocab_size=6이면 6x6의 항등 행렬)
# 예: np.eye(6)[1] → [0. 1. 0. 0. 0. 0.] (인덱스 1 위치만 1이고 나머지 0)

# x_data는 [[1, 4, 4, 3, 2]] 형태이므로,
# np.eye(vocab_size)[x]는 아래처럼 각 인덱스를 원-핫 인코딩 벡터로 변환함:
# [1, 4, 4, 3, 2] → [
#   [0. 1. 0. 0. 0. 0.],
#   [0. 0. 0. 0. 1. 0.],
#   [0. 0. 0. 0. 1. 0.],
#   [0. 0. 0. 1. 0. 0.],
#   [0. 0. 1. 0. 0. 0.]
# ]

# 이 변환을 x_data에 대해 반복 (배치 차원 유지)
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

# 결과 확인 (list of 2D array)
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [7]:
# 파이썬 리스트(x_one_hot)를 float 타입의 파이토치 텐서로 변환
# shape: (1, 5, 6) → 1개의 시퀀스, 시퀀스 길이 5, 원-핫 인코딩 길이 6
X = torch.FloatTensor(x_one_hot)

# 정답 레이블(y_data)을 long 타입의 파이토치 텐서로 변환
# CrossEntropyLoss에서는 target은 long 타입이어야 하며, softmax를 적용하지 않은 raw logit을 기대함
# shape: (1, 5) → 1개의 시퀀스, 시퀀스 길이 5
Y = torch.LongTensor(y_data)

# 변환된 텐서 출력: 실제 값과 차원을 확인
print(X)          # 입력 데이터 (원-핫 벡터로 변환된)
print(Y)          # 정답 레이블 (정수 인덱스 형태)
print(X.size(), Y.size())  # X: torch.Size([1, 5, 6]), Y: torch.Size([1, 5])

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])
torch.Size([1, 5, 5]) torch.Size([1, 5])


C:\Users\admin\AppData\Local\Temp\ipykernel_9120\2588651093.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X = torch.FloatTensor(x_one_hot)


In [8]:
# PyTorch의 기본 신경망 클래스(nn.Module)를 상속하여 Net 클래스 정의
class Net(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()  # 부모 클래스 초기화

        # RNN 모듈 생성
        # input_size : 각 글자가 갖는 벡터의 차원 (원-핫 인코딩 크기)
        # hidden_size : RNN 내부 상태(hidden state)의 차원
        # batch_first=True : 입력 텐서의 차원을 (batch_size, sequence_length, input_size)로 처리
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)

        # RNN의 출력(hidden state)을 받아 최종 예측 결과를 만들어주는 fully-connected layer
        # hidden_size → output_size 로 변환 (클래스 개수만큼 출력)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        # x: 입력 시퀀스, shape = (batch_size, seq_len, input_size)

        # RNN을 통해 시퀀스 전체에 대해 hidden state 계산
        # x: 모든 시점의 hidden state 출력 (shape = [batch_size, seq_len, hidden_size])
        # _status: 최종 hidden state (사용하지 않음, 그래서 _로 받음)
        x, _status = self.rnn(x)

        # 각 시점의 hidden state에 FC 레이어를 적용하여 클래스별 점수(logit) 출력
        # 최종 출력 shape = (batch_size, seq_len, output_size)
        x = self.fc(x)

        # 출력 결과 반환
        return x

In [9]:
# RNN 모델(Net 클래스의 인스턴스)을 생성하고,
# input_size, hidden_size, output_size는 이전에 정의한 값들임
net = Net(input_size, hidden_size, output_size)

# 손실 함수로 CrossEntropyLoss 사용
# - 다중 클래스 분류 문제에서 자주 사용됨
# - 내부적으로 softmax + log + NLLLoss까지 처리해줌
crit = nn.CrossEntropyLoss()

# 옵티마이저로 Adam 사용 (모델의 학습 가능한 파라미터에 대해)
# - learning_rate는 앞에서 0.1로 설정했음
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# ───────────────────────────────────────────────
# RNN에 입력 데이터를 넣고 출력값(output)을 받음
# 입력 X의 shape: (batch_size=1, seq_len=5, input_size=5)
output = net(X)

# ───────────────────────────────────────────────
# argmax(dim=-1)은 output 텐서의 마지막 차원(=클래스 차원)에서 가장 높은 값을 가진 인덱스를 반환
# 예) [0.1, 0.2, 0.05, 0.6, 0.05] → argmax = 3 (가장 높은 값의 인덱스)
# output.shape = (1, 5, 5) → 시퀀스 길이 5, 각 시점마다 5개 클래스에 대한 확률
print(output)
print(output.argmax(-1))  # 예측 결과: 각 시점에서 가장 높은 클래스 인덱스를 리스트로 반환

# 출력 텐서의 shape 확인
# → (batch_size, seq_len, output_size) = (1, 5, 5)
# → 1개 문장, 5글자, 각 글자마다 5개 클래스(문자) 중 하나를 예측
print(output.size())

tensor([[[-0.0756, -0.6618, -0.2945,  0.3018,  0.0715],
         [ 0.0695, -0.5252, -0.1642,  0.4059,  0.0200],
         [ 0.1103, -0.5232, -0.1931,  0.3370,  0.0388],
         [-0.0535, -0.3060, -0.0164,  0.1601,  0.0928],
         [ 0.3267, -0.4398, -0.0510,  0.1904, -0.0912]]],
       grad_fn=<ViewBackward0>)
tensor([[3, 3, 3, 3, 0]])
torch.Size([1, 5, 5])


In [10]:
for epoch in range(100):  # 총 100번 학습 반복 (에폭 = 전체 데이터를 1번 학습하는 단위)
    
    optimizer.zero_grad()  # ① 이전 단계에서 계산된 기울기(gradient)를 모두 0으로 초기화

    output = net(X)  # ② 현재 입력 데이터 X를 RNN 모델에 통과시켜 예측값 출력
                     #    output shape: (1, 5, 5) = (batch_size, seq_len, num_classes)

    # ③ 손실 계산
    # - CrossEntropyLoss는 (N, C)와 (N,) 형태를 입력으로 받음
    # - 그래서 output은 (1, 5, 5) → (5, 5)로 펼치고
    # - 정답 Y는 (1, 5) → (5,)로 펼쳐서 비교
    loss = crit(output.view(-1, input_size), Y.view(-1))  
    
    loss.backward()  # ④ 역전파: 손실을 기준으로 각 파라미터에 대한 기울기 계산

    optimizer.step()  # ⑤ 계산된 기울기를 바탕으로 파라미터 업데이트 (학습)

    # ─────────────────────────────────────
    # 결과 확인 (예측된 문자열 확인)
    # ─────────────────────────────────────

    # output은 텐서이므로 .data.numpy()로 NumPy 배열로 변환
    # axis=2: 클래스 차원(5개 클래스)에서 가장 큰 값의 인덱스를 추출
    # 결과: shape (1, 5) — 예측된 정답 인덱스
    result = output.data.numpy().argmax(axis=2)

    # 예측된 정답 인덱스를 대응되는 문자로 변환
    # - np.squeeze(result): (1,5) → (5,) 차원 축소
    # - index_to_char[i]로 문자 매핑
    str_result = ''.join([index_to_char[i] for i in np.squeeze(result)])

    # 예측 결과와 실제 정답 출력
    print(f'{epoch}, loss : {loss.item()}, prediction : {result}, true Y : {y_data}, prediction_str : {str_result}')

0, loss : 1.4475301504135132, prediction : [[3 3 3 3 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : llll!
1, loss : 1.2202279567718506, prediction : [[4 4 4 4 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pppp!
2, loss : 1.024251103401184, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pppe!
3, loss : 0.84978848695755, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pppe!
4, loss : 0.7186065912246704, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pppe!
5, loss : 0.5801458358764648, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pppe!
6, loss : 0.4321623742580414, prediction : [[4 4 3 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pple!
7, loss : 0.3116663694381714, prediction : [[4 4 3 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pple!
8, loss : 0.22664812207221985, prediction : [[4 4 3 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction_str : pple!
9, loss : 0.161177769

- 이번에는 조금 더 긴 문자를 이용해서 다음 문장을 예측하기

In [11]:
str = """if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."""
sentence = (str)
str

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [12]:
char_set = list(set(sentence))
print(len(char_set))
print(char_set)

25
['k', 'u', 'i', 'p', 'n', 'm', 'e', 'o', 'g', 'f', 'h', 'r', 'y', 'w', 'a', 't', ',', 'd', '.', 'l', ' ', 'c', 's', 'b', "'"]


In [13]:
char_dic = {c:i for i, c in enumerate(char_set)}
print(char_dic)
dic_size = len(char_set)
print(dic_size)


{'k': 0, 'u': 1, 'i': 2, 'p': 3, 'n': 4, 'm': 5, 'e': 6, 'o': 7, 'g': 8, 'f': 9, 'h': 10, 'r': 11, 'y': 12, 'w': 13, 'a': 14, 't': 15, ',': 16, 'd': 17, '.': 18, 'l': 19, ' ': 20, 'c': 21, 's': 22, 'b': 23, "'": 24}
25


In [14]:
# 하이퍼파라미터
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1
print(hidden_size)


25


In [15]:
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
    print(f'{x_str} : {y_str}')


if you wan : f you want
f you want :  you want 
 you want  : you want t
you want t : ou want to
ou want to : u want to 
u want to  :  want to b
 want to b : want to bu
want to bu : ant to bui
ant to bui : nt to buil
nt to buil : t to build
t to build :  to build 
 to build  : to build a
to build a : o build a 
o build a  :  build a s
 build a s : build a sh
build a sh : uild a shi
uild a shi : ild a ship
ild a ship : ld a ship,
ld a ship, : d a ship, 
d a ship,  :  a ship, d
 a ship, d : a ship, do
a ship, do :  ship, don
 ship, don : ship, don'
ship, don' : hip, don't
hip, don't : ip, don't 
ip, don't  : p, don't d
p, don't d : , don't dr
, don't dr :  don't dru
 don't dru : don't drum
don't drum : on't drum 
on't drum  : n't drum u
n't drum u : 't drum up
't drum up : t drum up 
t drum up  :  drum up p
 drum up p : drum up pe
drum up pe : rum up peo
rum up peo : um up peop
um up peop : m up peopl
m up peopl :  up people
 up people : up people 
up people  : p people t
p people t :  pe

In [16]:
print(x_data)
print(y_data)

[[2, 9, 20, 12, 7, 1, 20, 13, 14, 4], [9, 20, 12, 7, 1, 20, 13, 14, 4, 15], [20, 12, 7, 1, 20, 13, 14, 4, 15, 20], [12, 7, 1, 20, 13, 14, 4, 15, 20, 15], [7, 1, 20, 13, 14, 4, 15, 20, 15, 7], [1, 20, 13, 14, 4, 15, 20, 15, 7, 20], [20, 13, 14, 4, 15, 20, 15, 7, 20, 23], [13, 14, 4, 15, 20, 15, 7, 20, 23, 1], [14, 4, 15, 20, 15, 7, 20, 23, 1, 2], [4, 15, 20, 15, 7, 20, 23, 1, 2, 19], [15, 20, 15, 7, 20, 23, 1, 2, 19, 17], [20, 15, 7, 20, 23, 1, 2, 19, 17, 20], [15, 7, 20, 23, 1, 2, 19, 17, 20, 14], [7, 20, 23, 1, 2, 19, 17, 20, 14, 20], [20, 23, 1, 2, 19, 17, 20, 14, 20, 22], [23, 1, 2, 19, 17, 20, 14, 20, 22, 10], [1, 2, 19, 17, 20, 14, 20, 22, 10, 2], [2, 19, 17, 20, 14, 20, 22, 10, 2, 3], [19, 17, 20, 14, 20, 22, 10, 2, 3, 16], [17, 20, 14, 20, 22, 10, 2, 3, 16, 20], [20, 14, 20, 22, 10, 2, 3, 16, 20, 17], [14, 20, 22, 10, 2, 3, 16, 20, 17, 7], [20, 22, 10, 2, 3, 16, 20, 17, 7, 4], [22, 10, 2, 3, 16, 20, 17, 7, 4, 24], [10, 2, 3, 16, 20, 17, 7, 4, 24, 15], [2, 3, 16, 20, 17, 7, 4, 24

In [17]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
print(x_one_hot)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X.size(), Y.size())

[array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [18]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        # _를 먼저 쓰면 이후에 안쓸 변수다라는 것을 이야기 해줌
        x, _status = self.rnn(x)
        x=self.fc(x)
        return x
        

In [19]:
net = Net(dic_size, hidden_size, 2) # layer 2층을 쌓겠다는 의미
crit = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


In [20]:
output = net(X)
aa = output.argmax(dim=2)
print(aa)
print(aa.size())
print(output.size())
print(output.view(-1, dic_size).shape)

tensor([[14,  0,  0,  ..., 18,  0, 18],
        [14,  3,  0,  ...,  0, 18, 18],
        [14,  3, 11,  ..., 18, 18,  0],
        ...,
        [ 0,  3,  3,  ..., 14,  0, 18],
        [14,  3,  0,  ...,  0, 18, 14],
        [14,  0,  3,  ..., 18, 14,  0]])
torch.Size([170, 10])
torch.Size([170, 10, 25])
torch.Size([1700, 25])


In [21]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [22]:
for i in range(100):
    optimizer.zero_grad()
    output=net(X)
    loss = crit(output.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    results = output.argmax(dim=2)
    pred_str=''

    for j, result in enumerate(results):
        if j==0:
            pred_str +=''.join([char_set[t] for t in result])
        else:
            pred_str += char_set[result[-1]]
    print(pred_str)

akkk.kk.k..kk.kpkkkkkp..k,kkpk.kp.kpkkkapkkpapk.akk.ka.kakkk.kpakkakkkp.kk.p.kkp.kp.kk..kkr.kka.eke.kkkk..kpkkkkkpkk.kkkkakkkak.kkkka.ek.k..kka..kpkkakpk.ka..kk.eapekkkkpkpkkak.ak
                                                                                                                                                                                   
teeenneelnnnennnnnnnnennnnnnennnnnnnnnennnennnnnnnnnnnnennneennennneennennnnnnnennennennennnennnnnnnnennnnnnnnnnnnnnennnnnnnennnennnnnnnennennnnennnnenneennennnnnennnnnnnnnennneen
tbbdtobbt t  t      t       o t  oo   o   t            o   o        ot       t o           t     o  t     t                t                       oo   ot  o               o      
tdrmotduothtttbhttttthttthdttshhdttthtttthttthhh httthtttohttthtthottthtttthttthttthtthhtttttttdttthhtthhhthtthhttthttdhtttttthhtthhthhttthtthhttthttthhttohtthh ttttttttthttttdhtt
 dr t to  tr d trdrrfrdrd drdri d dh rde rffbir rbdrr  dr fdt  rhr t d tdtr dfbrdhdrdrdrdh f r dthr 

In [24]:
result = net(X[5])
result1 = result.argmax(-1)
str=''.join([char_set[i] for i in result1])
print(str)

mwant to b
